# Assignment 2

Reading the text and using sentence tokenizer

In [123]:
import re
with open("C:/Users/Mohit Mina/Downloads/speeches.txt","r",encoding="utf-8") as file:
    text=file.read()
    text=text.lower()
    text=re.sub('\n','',text)
from nltk.tokenize import sent_tokenize
text = sent_tokenize(text)
print(len(text))
text

12601


['\ufeffspeech 1...thank you so much.',
 "that's so nice.",
 "isn't he a great guy.",
 "he doesn't get a fair press; he doesn't get it.",
 "it's just not fair.",
 "and i have to tell you i'm here, and very strongly here, because i have great respect for steve king and have great respect likewise for citizens united, david and everybody, and tremendous resect for the tea party.",
 'also, also the people of iowa.',
 'they have something in common.',
 'hard-working people.',
 'they want to work, they want to make the country great.',
 'i love the people of iowa.',
 "so that's the way it is.",
 'very simple.with that said, our country is really headed in the wrong direction with a president who is doing an absolutely terrible job.',
 "the world is collapsing around us, and many of the problems we've caused.",
 'our president is either grossly incompetent, a word that more and more people are using, and i think i was the first to use it, or he has a completely different agenda than you want

In [124]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(text, test_size = 0.2)

print(len(train))
print(len(test))
print(train)

10080
2521
['enjoy yourself.', 'they said, "why don’t you skip iowa?"', 'but they have a whole big thing on eminent domain in their prospectus because without eminent domain, they wouldn’t go 15 feet.', 'we are going to help the country, that’s called helping the daughter.', 'we have nothing.', 'stand up.', 'they said, mr. trump, the wall is vital.', 'but you know what?', 'that deal was signed by bill clinton.', 'we have such an amazing relationship with hispanics and they’re great people.the problem is the leaders of mexico are too smart for our leaders.', 'but the evangelicals left me down a little bit just last month.', 'the waste that i get rid of – the waste that i get rid of – is going to have a huge impact and i’m not even putting that in my numbers so i think it’s going to be terrific.', 'we are getting rid of common core.', 'remember that – low debt, great cash flow, and the greatest assets – doral, trump tower, the trump building, big chunk of the bank of america building in 

In [125]:
import string
train_set=[]
for i in train:
    i = re.sub(r'[^\w\s]','',i)
    train_set.append(i)
print(len(train_set))
train_set        

10080


['enjoy yourself',
 'they said why dont you skip iowa',
 'but they have a whole big thing on eminent domain in their prospectus because without eminent domain they wouldnt go 15 feet',
 'we are going to help the country thats called helping the daughter',
 'we have nothing',
 'stand up',
 'they said mr trump the wall is vital',
 'but you know what',
 'that deal was signed by bill clinton',
 'we have such an amazing relationship with hispanics and theyre great peoplethe problem is the leaders of mexico are too smart for our leaders',
 'but the evangelicals left me down a little bit just last month',
 'the waste that i get rid of  the waste that i get rid of  is going to have a huge impact and im not even putting that in my numbers so i think its going to be terrific',
 'we are getting rid of common core',
 'remember that  low debt great cash flow and the greatest assets  doral trump tower the trump building big chunk of the bank of america building in san francisco 1290 ave land all ove

In [126]:
import nltk
from nltk import word_tokenize
tokens=[]
for i in range(len(train_set)):
    tokens.append('<s>')
    tokens_of_sent=word_tokenize(train_set[i])
    for j in range(len(tokens_of_sent)):
        tokens.append(tokens_of_sent[j])
    tokens.append('</s>')
tokens

['<s>',
 'enjoy',
 'yourself',
 '</s>',
 '<s>',
 'they',
 'said',
 'why',
 'dont',
 'you',
 'skip',
 'iowa',
 '</s>',
 '<s>',
 'but',
 'they',
 'have',
 'a',
 'whole',
 'big',
 'thing',
 'on',
 'eminent',
 'domain',
 'in',
 'their',
 'prospectus',
 'because',
 'without',
 'eminent',
 'domain',
 'they',
 'wouldnt',
 'go',
 '15',
 'feet',
 '</s>',
 '<s>',
 'we',
 'are',
 'going',
 'to',
 'help',
 'the',
 'country',
 'thats',
 'called',
 'helping',
 'the',
 'daughter',
 '</s>',
 '<s>',
 'we',
 'have',
 'nothing',
 '</s>',
 '<s>',
 'stand',
 'up',
 '</s>',
 '<s>',
 'they',
 'said',
 'mr',
 'trump',
 'the',
 'wall',
 'is',
 'vital',
 '</s>',
 '<s>',
 'but',
 'you',
 'know',
 'what',
 '</s>',
 '<s>',
 'that',
 'deal',
 'was',
 'signed',
 'by',
 'bill',
 'clinton',
 '</s>',
 '<s>',
 'we',
 'have',
 'such',
 'an',
 'amazing',
 'relationship',
 'with',
 'hispanics',
 'and',
 'theyre',
 'great',
 'peoplethe',
 'problem',
 'is',
 'the',
 'leaders',
 'of',
 'mexico',
 'are',
 'too',
 'smart',
 'fo

In [127]:
from collections import Counter
from nltk.util import ngrams
unigram=tokens
list(unigram)

['<s>',
 'enjoy',
 'yourself',
 '</s>',
 '<s>',
 'they',
 'said',
 'why',
 'dont',
 'you',
 'skip',
 'iowa',
 '</s>',
 '<s>',
 'but',
 'they',
 'have',
 'a',
 'whole',
 'big',
 'thing',
 'on',
 'eminent',
 'domain',
 'in',
 'their',
 'prospectus',
 'because',
 'without',
 'eminent',
 'domain',
 'they',
 'wouldnt',
 'go',
 '15',
 'feet',
 '</s>',
 '<s>',
 'we',
 'are',
 'going',
 'to',
 'help',
 'the',
 'country',
 'thats',
 'called',
 'helping',
 'the',
 'daughter',
 '</s>',
 '<s>',
 'we',
 'have',
 'nothing',
 '</s>',
 '<s>',
 'stand',
 'up',
 '</s>',
 '<s>',
 'they',
 'said',
 'mr',
 'trump',
 'the',
 'wall',
 'is',
 'vital',
 '</s>',
 '<s>',
 'but',
 'you',
 'know',
 'what',
 '</s>',
 '<s>',
 'that',
 'deal',
 'was',
 'signed',
 'by',
 'bill',
 'clinton',
 '</s>',
 '<s>',
 'we',
 'have',
 'such',
 'an',
 'amazing',
 'relationship',
 'with',
 'hispanics',
 'and',
 'theyre',
 'great',
 'peoplethe',
 'problem',
 'is',
 'the',
 'leaders',
 'of',
 'mexico',
 'are',
 'too',
 'smart',
 'fo

In [128]:
c_unigram=len(unigram)
bigrams = list(nltk.bigrams(tokens))
trigrams = list(nltk.trigrams(tokens))
quadgrams = list(ngrams(tokens,4))
print("number of possible unigrams = ",c_unigram)
print("actual number of unigrams =",c_unigram)

print("number of possible bigrams = ",c_unigram**2)
print("actual number of bigrams =",len(bigrams))

print("number of possible trigrams = ",c_unigram**3)
print("actual number of trigrams =",len(trigrams))

print("number of possible quadgrams = ",c_unigram**4)
print("actual number of quadgrams =",len(quadgrams))

number of possible unigrams =  149771
actual number of unigrams = 149771
number of possible bigrams =  22431352441
actual number of bigrams = 149770
number of possible trigrams =  3359566086441011
actual number of trigrams = 149769
number of possible quadgrams =  503165572332356658481
actual number of quadgrams = 149768


In [129]:
def MLE_unigrams(word):
    prob=unigram.count(word)/(c_unigram)
    return prob
print(MLE_unigrams('look'))

0.0019496431218326644


In [130]:
def MLE_bigrams(word):
    prob=bigrams.count(word)/(unigram.count(word[0]))
    return prob
print(MLE_bigrams(('be','elected')))

0.0010787486515641855


In [131]:
def MLE_trigrams(word):
    prob=trigrams.count(word)/(bigrams.count(word[:2]))
    return prob
print(MLE_trigrams(('shouldnt','have','done')))

0.1


In [132]:
def MLE_quadgrams(word):
    prob=quadgrams.count(word)/(trigrams.count(word[:3]))
    return prob
print(MLE_quadgrams(('way','its','going','to')))

0.6666666666666666


In [ ]:
import numpy as np

def bigrams_generator():
    list_=[]
    sentence="<s>"
    start='<s>'
    while(1):
        for i in bigrams:
            if i[0]==start:
                list_.append(i)
        prob_list=[MLE_bigrams(i) for i in list_]
        sum_of_prob=sum(prob_list)
        prob_list=[i/sum_of_prob for i in prob_list]          #prob(i)/sum of prob 
        ans = list(np.random.multinomial(1,prob_list))
        ind = np.argmax(ans)
        new_word = list_[ind][1]
        sentence=sentence+" "+new_word
        if (new_word=='</s>'):
            break
        else:
            start = new_word
    return sentence

In [171]:
def generate(k):
    if k==1:
        return bigrams_generator()
    elif k==2:
        return trigrams_generator()
    elif k==3:
        return quadgrams_generator()
    else:
        return "Model no. not valid"

In [156]:
import string
test_set=[]
for i in test:
    i = re.sub(r'[^\w\s]','',i)
    test_set.append(i)
print(len(test_set))
test_set  

2521


['were winning on education',
 'you know its supposed to be free trade',
 'theyd leave  today theyd leave this country for other countriesso now were losing pfizer',
 'except for the times',
 'and i said if you go after one or the other in this case iraq youre going to destabilize the middle east',
 'well have plenty  well talk a lot about isis',
 'and i want to thank my great parents who are looking down right now and my brother looking down on us',
 'these are people with carrier for many many years',
 'the level of genius  they fully understand',
 'they want that  they want education',
 'im endorsing you',
 'he gave 100the liberal media is the only ones that have been calling the foundation',
 'i havent  i never really got to read it to be honest',
 'we love you',
 'i said what are you doing',
 'the decision to overthrow the regime in libya then pushing for the overthrow of the regime in syria among other things without plans for the day after have created space for isis to expand a

In [157]:
import nltk
from nltk import word_tokenize
tokens=[]
for i in range(len(train_set)):
    tokens.append('<s>')
    tokens_of_sent=word_tokenize(train_set[i])
    for j in range(len(tokens_of_sent)):
        tokens.append(tokens_of_sent[j])
    tokens.append('</s>')
tokens

['<s>',
 'enjoy',
 'yourself',
 '</s>',
 '<s>',
 'they',
 'said',
 'why',
 'dont',
 'you',
 'skip',
 'iowa',
 '</s>',
 '<s>',
 'but',
 'they',
 'have',
 'a',
 'whole',
 'big',
 'thing',
 'on',
 'eminent',
 'domain',
 'in',
 'their',
 'prospectus',
 'because',
 'without',
 'eminent',
 'domain',
 'they',
 'wouldnt',
 'go',
 '15',
 'feet',
 '</s>',
 '<s>',
 'we',
 'are',
 'going',
 'to',
 'help',
 'the',
 'country',
 'thats',
 'called',
 'helping',
 'the',
 'daughter',
 '</s>',
 '<s>',
 'we',
 'have',
 'nothing',
 '</s>',
 '<s>',
 'stand',
 'up',
 '</s>',
 '<s>',
 'they',
 'said',
 'mr',
 'trump',
 'the',
 'wall',
 'is',
 'vital',
 '</s>',
 '<s>',
 'but',
 'you',
 'know',
 'what',
 '</s>',
 '<s>',
 'that',
 'deal',
 'was',
 'signed',
 'by',
 'bill',
 'clinton',
 '</s>',
 '<s>',
 'we',
 'have',
 'such',
 'an',
 'amazing',
 'relationship',
 'with',
 'hispanics',
 'and',
 'theyre',
 'great',
 'peoplethe',
 'problem',
 'is',
 'the',
 'leaders',
 'of',
 'mexico',
 'are',
 'too',
 'smart',
 'fo

# Perplexity

In [168]:
import math
def log_prob(sentence,k):
    prob=0
    if k == 0:
        prob = 0
        for i in range(len(sentence)):
            try:
                prob += math.log(MLE_unigrams(sentence[i]))
            except:
                continue
        return prob
    elif k == 1:
        try:
            prob = math.log(MLE_unigrams(sentence[0]))
        except:
            pass
        for i in range(len(sentence)-1):
            try:
                prob += math.log(MLE_bigrams(sentence[i:i+2]))
            except:
                continue
        return prob
    
    elif k == 2:
        try:    
            prob = math.log(MLE_unigrams(sentence[0]))+math.log(MLE_bigrams(sentence[0:2]))
        except:
            pass
        for i in range(len(sentence)-2):
            try:
                prob += math.log(MLE_trigrams(sentence[i:i+3]))
            except:
                continue        
        return prob
    
    elif k == 3:
        try:    
            prob = math.log(MLE_unigrams(sentence[0]))+math.log(MLE_bigrams(sentence[0:2]))+math.log(MLE_trigrams(sentence[0:3]))
        except:
            pass
        for i in range(len(sentence)-3):
            try:
                prob += math.log(MLE_quadgrams(sentence[i:i+4]))
            except:
                continue
        return prob


In [170]:
perp = math.exp(-1*log_prob(test,3)/len(test))
print ("Perplexity : ",perp)